In [109]:
!pip install mechanize htmldom slack_sdk

In [10]:
import mechanize
import credentials
import re
import pickle
from difflib import SequenceMatcher

In [15]:
if True:
    br = mechanize.Browser()
    br.addheaders = [('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/114.0')]
    br.open("https://www.show-score.com/users/sign_in")

    br.select_form(nr=0)

    br.find_control(name="user[email]").value = credentials.USERNAME
    br.find_control(name="user[password]").value = credentials.PASSWORD

    response_login = br.submit()

    html = response_login.get_data()
    #print html


    br.open("https://www.show-score.com/promotions")
    html = br.response().read()

In [20]:
from html.parser import HTMLParser
from htmldom import htmldom

In [27]:
dom = htmldom.HtmlDom()
dom.createDom(htmlString=html.decode())

In [83]:
new_avail  = []

In [84]:
show_list = dom.find('.show-details__header')

In [85]:
for a_show in show_list:
    title = a_show.children().filter('.show-details__show-title-header-link').text().strip()
    location = a_show.children().filter('.show-details__location-name').text().strip()
    new_avail.append(title + ' // ' + location)

In [86]:
new_avail.sort()

In [87]:
print(new_avail)

['A Tale of Two Cities, Cobbled Together by the Brothers Lovejoy Member Nights // NJ: Cape May', 'Back and Forth Member Nights // NYC: Upper E Side', 'Eisenhower: This Piece of Ground Member Nights // NYC: Midtown W', 'Here Lies Love (Broadway) Member Nights // NYC: Midtown W', 'Notre Dame de Paris Member Nights // NYC: Upper W Side', 'The Doctor (Park Avenue Armory) Member Nights // NYC: Upper E Side', 'The Rise and Fall of Jean Claude Van Damme Member Nights // NYC: Brooklyn', 'Triple Threat Member Nights // NYC: Midtown W']


In [88]:
def differ(old, new):
    new_offerings = []
    for new_candidate in new:
        #print "======================"
        #print "Looking for match for " + new_candidate
        old_matching_entry = list(filter(lambda x: SequenceMatcher(None, x, new_candidate).ratio() > 0.8, old))
        if not old_matching_entry:
            new_offerings.append(new_candidate)
            #print "DIFFER IS SAYING THIS IS NEW:"
            #print new_candidate
            #print old
        #else:
            #print "Found match: "
            #print old_matching_entry
    return new_offerings

In [103]:
def fetch_previous_available_titles():
    with open("avail_titles.pkl", "rb") as f:
        return pickle.load(f)

In [104]:
fetch_previous_available_titles()

['A Tale of Two Cities, Cobbled Together by the Brothers Lovejoy Member Nights // NJ: Cape May',
 'Back and Forth Member Nights // NYC: Upper E Side',
 'Eisenhower: This Piece of Ground Member Nights // NYC: Midtown W',
 'Here Lies Love (Broadway) Member Nights // NYC: Midtown W',
 'Notre Dame de Paris Member Nights // NYC: Upper W Side',
 'The Doctor (Park Avenue Armory) Member Nights // NYC: Upper E Side',
 'The Rise and Fall of Jean Claude Van Damme Member Nights // NYC: Brooklyn',
 'Triple Threat Member Nights // NYC: Midtown W']

In [105]:
prev_avail = None
try:
    prev_avail = fetch_previous_available_titles()
except:
    prev_avail = []

In [106]:
prev_avail

['A Tale of Two Cities, Cobbled Together by the Brothers Lovejoy Member Nights // NJ: Cape May',
 'Back and Forth Member Nights // NYC: Upper E Side',
 'Eisenhower: This Piece of Ground Member Nights // NYC: Midtown W',
 'Here Lies Love (Broadway) Member Nights // NYC: Midtown W',
 'Notre Dame de Paris Member Nights // NYC: Upper W Side',
 'The Doctor (Park Avenue Armory) Member Nights // NYC: Upper E Side',
 'The Rise and Fall of Jean Claude Van Damme Member Nights // NYC: Brooklyn',
 'Triple Threat Member Nights // NYC: Midtown W']

In [107]:
new_avail

['A Tale of Two Cities, Cobbled Together by the Brothers Lovejoy Member Nights // NJ: Cape May',
 'Back and Forth Member Nights // NYC: Upper E Side',
 'Eisenhower: This Piece of Ground Member Nights // NYC: Midtown W',
 'Here Lies Love (Broadway) Member Nights // NYC: Midtown W',
 'Notre Dame de Paris Member Nights // NYC: Upper W Side',
 'The Doctor (Park Avenue Armory) Member Nights // NYC: Upper E Side',
 'The Rise and Fall of Jean Claude Van Damme Member Nights // NYC: Brooklyn',
 'Triple Threat Member Nights // NYC: Midtown W']

In [108]:
new_offerings = differ(prev_avail, new_avail)
new_offerings

[]

In [110]:
for offering in new_offerings:
    from slack_sdk.webhook import WebhookClient
    url = 'https://hooks.slack.com/services/%s' % credentials.SLACK
    webhook = WebhookClient(url)
    retval = webhook.send(text=str(offering))
    print("Return value from the slack webhook launch: " + str(retval))

In [98]:
if new_offerings:
    with open("avail_titles.pkl", "wb") as f:
        pickle.dump(new_avail, f)